In [1]:
import numpy as np
import pandas as pd

data=pd.read_csv("./data/train.csv")
textf=np.load("./data/text_features.npy")
imgf=np.load("./data/image_features.npy")

In [2]:
from keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=data,
    directory="./data/train_images/",
    x_col="image",
    y_col=None,
    batch_size=8,
    shuffle=False,
    seed=123,
    class_mode=None,
    target_size=(229,229))

Using TensorFlow backend.
/home/shreyas/.local/lib/python3.6/site-packages/keras_preprocessing/image.py:2059: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.df[x_col] = self.df[x_col].astype(str)


Found 15643 images.


In [3]:
filenames = test_generator.filenames

In [7]:
images_features=[]
for image in data['image']:
    index=filenames.index(image)
    images_features.append(imgf[index])

In [8]:
images_features=np.array(images_features)

In [9]:
images_features.shape

(24396, 2048)

In [10]:
textf.shape

(24396, 3708)

In [11]:
X=np.concatenate([images_features,textf],axis=1)

In [12]:
X.shape

(24396, 5756)

In [14]:
y=data['answer'].replace({'yes':1,'no':0}).tolist()
from keras.utils import to_categorical
y=to_categorical(y)

In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=123)

In [19]:
X_train.shape

(17077, 5756)

In [20]:
X_test.shape

(7319, 5756)

In [23]:
from keras.layers import Dense,Input
from keras.models import Model

def create_model(feature_size=5756,classes=2):
    inp=Input((feature_size,))
    X=Dense(units=500,activation='relu',use_bias=True)(inp)
    output=Dense(units=2,activation='softmax')(X)
    return Model(inp,output)
model=create_model()

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 5756)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               2878500   
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 1002      
Total params: 2,879,502
Trainable params: 2,879,502
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X=X_train,y=y_train,epochs=10,verbose=1,shuffle=False)

In [ ]:
model.evaluate(X_train,y_train,verbose=1)